<a href="https://colab.research.google.com/github/Ahtesham-Ibne-Mostafa/Natural_Language_Processing/blob/main/Disease_Prediction_from_ehr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

ahteshamibnemostafa_mimic_iv_merged_path = kagglehub.dataset_download('ahteshamibnemostafa/mimic-iv-merged')

print('Data source import complete.')


In [ ]:
!nvidia-smi


Wed Jun 11 23:21:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directo           c  tttttttt

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "/kaggle/input/mimic-iv-merged/processed_dataset.csv"

# Load the latest version
df = pd.read_csv(file_path)
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas

In [ ]:
df.head()

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,note_id,input,target,input_tokens,target_tokens,hadm_id,note_type,note_seq,charttime,storetime
0,10000032,F,52,2180,2014 - 2016,2180-09-09,10000032-DS-21,<SEX> F <SERVICE> MEDICINE <ALLERGIES> No Know...,"___ HCV cirrhosis c/b ascites, hiv on ART, h/o...",1946,231,22595853,DS,21,2180-05-07 00:00:00,2180-05-09 15:26:00
1,10000032,F,52,2180,2014 - 2016,2180-09-09,10000032-DS-22,<SEX> F <SERVICE> MEDICINE <ALLERGIES> Percoce...,"___ with HIV on HAART, HCV cirrhosis with asci...",2183,810,22841357,DS,22,2180-06-27 00:00:00,2180-07-01 10:15:00
2,10000117,F,48,2174,2008 - 2010,NaN,10000117-DS-21,<SEX> F <SERVICE> MEDICINE <ALLERGIES> omepraz...,Ms. ___ is a ___ with history of GERD who pres...,1060,172,22927623,DS,21,2181-11-15 00:00:00,2181-11-15 15:04:00
3,10000117,F,48,2174,2008 - 2010,NaN,10000117-DS-22,<SEX> F <SERVICE> ORTHOPAEDICS <ALLERGIES> ome...,The patient presented to the emergency departm...,1195,330,27988844,DS,22,2183-09-21 00:00:00,2183-09-29 16:23:00
4,10000248,M,34,2192,2014 - 2016,NaN,10000248-DS-10,<SEX> M <SERVICE> MEDICINE <ALLERGIES> No Know...,Mr. ___ is a ___ with history of mild FVIII de...,1961,230,20600184,DS,10,2192-11-30 00:00:00,2192-11-30 19:49:00


In [ ]:
df['dod'] = df['dod'].apply(lambda x: 1 if pd.notnull(x) and str(x).strip() != '' else 0)

In [ ]:
df = df.drop(columns=['subject_id','note_id','anchor_year','anchor_year_group','input_tokens','target_tokens','hadm_id','note_seq','charttime','storetime'])

In [ ]:
# Add metadata to input text
def augment_input(row):
    mortality = "Deceased" if row['dod'] == 0 else "Alive"
    return (
        f"<Age> {row['anchor_age']} "
        f"<Mortality> {mortality} "
        f"<Note Type> {row['note_type']} "
        f"{row['input']}"
    )

df['augmented_input'] = df.apply(augment_input, axis=1)

In [ ]:
df = df.drop(columns=[
'gender',	'anchor_age',	'dod',	'input',	'note_type'])

In [ ]:
df.head()

,target,augmented_input
0,"___ HCV cirrhosis c/b ascites, hiv on ART, h/o...",<Age> 52 <Mortality> Alive <Note Type> DS <SEX...
1,"___ with HIV on HAART, HCV cirrhosis with asci...",<Age> 52 <Mortality> Alive <Note Type> DS <SEX...
2,Ms. ___ is a ___ with history of GERD who pres...,<Age> 48 <Mortality> Deceased <Note Type> DS <...
3,The patient presented to the emergency departm...,<Age> 48 <Mortality> Deceased <Note Type> DS <...
4,Mr. ___ is a ___ with history of mild FVIII de...,<Age> 34 <Mortality> Deceased <Note Type> DS <...


In [ ]:
df

,target,augmented_input
0,"___ HCV cirrhosis c/b ascites, hiv on ART, h/o...",<Age> 52 <Mortality> Alive <Note Type> DS <SEX...
1,"___ with HIV on HAART, HCV cirrhosis with asci...",<Age> 52 <Mortality> Alive <Note Type> DS <SEX...
2,Ms. ___ is a ___ with history of GERD who pres...,<Age> 48 <Mortality> Deceased <Note Type> DS <...
3,The patient presented to the emergency departm...,<Age> 48 <Mortality> Deceased <Note Type> DS <...
4,Mr. ___ is a ___ with history of mild FVIII de...,<Age> 34 <Mortality> Deceased <Note Type> DS <...
...,...,...
270028,Mr. ___ is a ___ without significant medical h...,<Age> 57 <Mortality> Deceased <Note Type> DS <...
270029,Ms. ___ is a ___ year old female with a histor...,<Age> 46 <Mortality> Deceased <Note Type> DS <...
270030,___ year old man hx of epilepsy and left tempo...,<Age> 58 <Mortality> Alive <Note Type> DS <SEX...
270031,Mr. ___ was admitted with status epilepticus. ...,<Age> 58 <Mortality> Alive <Note Type> DS <SEX...


In [ ]:
df['target'][0]

'___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, bioplar, PTSD, presented from OSH ED with worsening abd distension over past week and confusion. # Ascites - p/w worsening abd distension and discomfort for last week. likely ___ portal HTN given underlying liver disease, though no ascitic fluid available on night of admission. No signs of heart failure noted on exam. This was ___ to med non-compliance and lack of diet restriction. SBP negative diuretics: > Furosemide 40 mg PO DAILY > Spironolactone 50 mg PO DAILY, chosen over the usual 100mg dose d/t K+ of 4.5. CXR was wnl, UA negative, Urine culture blood culture negative. Pt was losing excess fluid appropriately with stable lytes on the above regimen. Pt was scheduled with current PCP for ___ check upon discharge. Pt was scheduled for new PCP with Dr. ___ at ___ and follow up in Liver clinic to schedule outpatient screening EGD and ___. '

In [ ]:
df['augmented_input'][0]

'<Age> 52 <Mortality> Alive <Note Type> DS <SEX> F <SERVICE> MEDICINE <ALLERGIES> No Known Allergies / Adverse Drug Reactions <ATTENDING> ___ <CHIEF COMPLAINT> Worsening ABD distension and pain <MAJOR SURGICAL OR INVASIVE PROCEDURE> Paracentesis <HISTORY OF PRESENT ILLNESS> ___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, bioplar, PTSD, presented from OSH ED with worsening abd distension over past week. Pt reports self-discontinuing lasix and spirnolactone ___ weeks ago, because she feels like "they don\'t do anything" and that she "doesn\'t want to put more chemicals in her." She does not follow Na-restricted diets. In the past week, she notes that she has been having worsening abd distension and discomfort. She denies ___ edema, or SOB, or orthopnea. She denies f/c/n/v, d/c, dysuria. She had food poisoning a week ago from eating stale cake (n/v 20 min after food ingestion), which resolved the same day. She denies other recent illness or sick contacts. She notes that she has

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words
print(stop_words)


{'he', 'then', 'anything', 'therefore', 'the', 'serious', "'re", 'anywhere', 'next', 'very', 'done', 'few', 'thereafter', 'herein', 'is', 'get', 'various', 'elsewhere', 'too', 'also', 'thus', 'but', 'for', 'more', 'must', 'sometimes', 'have', 'except', 'not', 're', 'each', 'latterly', 'say', 'yours', 'seems', 'who', 'further', 'had', 'full', 'somehow', 'whole', 'someone', 'its', 'at', 'sixty', 'many', 'there', 'go', 'on', 'because', 'by', 'six', '’m', '’s', 'themselves', 'be', 'something', 'either', 'bottom', 'alone', 'really', 'a', 'neither', 'five', 'an', 'himself', 'herself', 'three', 'which', 'amount', 'their', 'up', 'everywhere', '‘ll', 'did', 'enough', 'nothing', 'become', 'much', 'via', 'do', 'amongst', 'and', 'beforehand', '’ve', 'already', 'from', 'has', 'your', 'fifteen', 'you', 'becomes', 'again', 'third', 'behind', 'four', 'been', 'last', 'almost', 'does', 'first', 'we', 'quite', 'never', 'two', 'anyhow', 'every', 'whereupon', 'that', 'along', 'else', 'myself', 'nine', 'les

In [ ]:
import string
punctuations = string.punctuation
print(punctuations)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [ ]:
import re
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Words we want to keep even though spaCy treats them as stopwords
keep_words = {"yes", "no"}

# Make sure "yes" and "no" are not marked as stopwords in the vocab
for word in keep_words:
    nlp.vocab[word].is_stop = False

# Get current stop words after editing
stop_words = STOP_WORDS - keep_words
punctuations = set(punctuation)

def clean_ehr_text(raw_text):
    # Define sections to REMOVE (regex is case-insensitive, ignores <...>)
    remove_sections = [
        r"<SEX>.*?(?=<|$)",
        r"<SERVICE>.*?(?=<|$)",
        r"<Age>.*?(?=<|$)",
        r"<Mortality>.*?(?=<|$)",
        r"<Note Type>.*?(?=<|$)",
        r"<ATTENDING>.*?(?=<|$)",
        r"<SOCIAL HISTORY>.*?(?=<|$)",
        r"<FAMILY HISTORY>.*?(?=<|$)",
        r"<DISCHARGE INSTRUCTIONS>.*?(?=<|$)",
        r"<DISCHARGE DISPOSITION>.*?(?=<|$)",
        r"<DISCHARGE DIAGNOSIS>.*?(?=<|$)",
        r"<DISCHARGE CONDITION>.*?(?=<|$)",
        r"<FOLLOWUP INSTRUCTIONS>.*?(?=<|$)",
        r"<DISCHARGE MEDICATIONS>.*?(?=<|$)",
        r"<PAST MEDICAL HISTORY>.*?(?=<|$)",
    ]

    combined_pattern = re.compile("|".join(remove_sections), re.IGNORECASE | re.DOTALL)
    cleaned_text = re.sub(combined_pattern, "", raw_text)

    # Remove excessive blank lines and whitespace
    cleaned_text = re.sub(r'\n\s*\n+', '\n\n', cleaned_text).strip()
    text = cleaned_text

    # Clean and format text
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"(</?[A-Z ]+>)", r"\n\1\n", text)
    text = re.sub(r"_+", "", text)
    text = re.sub(r"\d{1,2}:\d{2}\s*(?:AM|PM)?", "", text, flags=re.IGNORECASE)
    text = re.sub(r"[\.\:]\s*___", "", text)
    text = re.sub(r"(</?[A-Z ]+>)", lambda m: m.group(1).upper(), text)
    text = re.sub(r"\s+([,:;])", r"\1", text)
    text = re.sub(r"([,:;])\s+", r"\1 ", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"\n{2,}", "\n", text)

    # Tokenize and lemmatize using spaCy
    doc = nlp(text.strip())
    mytokens = [
        token.lemma_.lower().strip()
        for token in doc
        if (token.text.lower() in keep_words) or (not token.is_stop and token.text not in punctuations)
    ]

    return mytokens


raw_ehr = '<Age> 52 <Mortality> Alive <Note Type> DS <SEX> F <SERVICE> MEDICINE <ALLERGIES> No Known Allergies / Adverse Drug Reactions <ATTENDING> ___ <CHIEF COMPLAINT> Worsening ABD distension and pain <MAJOR SURGICAL OR INVASIVE PROCEDURE> Paracentesis <HISTORY OF PRESENT ILLNESS> ___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, bioplar, PTSD, presented from OSH ED with worsening abd distension over past week. Pt reports self-discontinuing lasix and spirnolactone ___ weeks ago, because she feels like "they don\'t do anything" and that she "doesn\'t want to put more chemicals in her." She does not follow Na-restricted diets. In the past week, she notes that she has been having worsening abd distension and discomfort. She denies ___ edema, or SOB, or orthopnea. She denies f/c/n/v, d/c, dysuria. She had food poisoning a week ago from eating stale cake (n/v 20 min after food ingestion), which resolved the same day. She denies other recent illness or sick contacts. She notes that she has been noticing gum bleeding while brushing her teeth in recent weeks. she denies easy bruising, melena, BRBPR, hemetesis, hemoptysis, or hematuria. Because of her abd pain, she went to OSH ED and was transferred to ___ for further care. Per ED report, pt has brief period of confusion - she did not recall the ultrasound or bloodwork at osh. She denies recent drug use or alcohol use. She denies feeling confused, but reports that she is forgetful at times. In the ED, initial vitals were 98.4 70 106/63 16 97%RA Labs notable for ALT/AST/AP ___ ___: ___, Tbili1.6, WBC 5K, platelet 77, INR 1.6 <PAST MEDICAL HISTORY> 1. HCV Cirrhosis 2. No history of abnormal Pap smears. 3. She had calcification in her breast, which was removed previously and per patient not, it was benign. 4. For HIV disease, she is being followed by Dr. ___ Dr. ___. 5. COPD 6. Past history of smoking. 7. She also had a skin lesion, which was biopsied and showed skin cancer per patient report and is scheduled for a complete removal of the skin lesion in ___ of this year. 8. She also had another lesion in her forehead with purple discoloration. It was biopsied to exclude the possibility of ___\'s sarcoma, the results is pending. 9. A 15 mm hypoechoic lesion on her ultrasound on ___ and is being monitored by an MRI. 10. History of dysplasia of anus in ___. 11. Bipolar affective disorder, currently manic, mild, and PTSD. 12. History of cocaine and heroin use. <SOCIAL HISTORY> ___ <FAMILY HISTORY> She a total of five siblings, but she is not talking to most of them. She only has one brother that she is in touch with and lives in ___. She is not aware of any known GI or liver disease in her family. Her last alcohol consumption was one drink two months ago. No regular alcohol consumption. Last drug use ___ years ago. She quit smoking a couple of years ago. <PHYSICAL EXAM> VS: 98.1 107/61 78 18 97RA General: in NAD HEENT: CTAB, anicteric sclera, OP clear Neck: supple, no LAD CV: RRR,S1S2, no m/r/g Lungs: CTAb, prolonged expiratory phase, no w/r/r Abdomen: distended, mild diffuse tenderness, +flank dullness, cannot percuss liver/spleen edge ___ distension GU: no foley Ext: wwp, no c/e/e, + clubbing Neuro: AAO3, converse normally, able to recall 3 times after 5 minutes, CN II-XII intact Discharge: PHYSICAL EXAMINATION: VS: 98 105/70 95 General: in NAD HEENT: anicteric sclera, OP clear Neck: supple, no LAD CV: RRR,S1S2, no m/r/g Lungs: CTAb, prolonged expiratory phase, no w/r/r Abdomen: distended but improved, TTP in RUQ, GU: no foley Ext: wwp, no c/e/e, + clubbing Neuro: AAO3, CN II-XII intact <PERTINENT RESULTS> ___ 10: 25PM GLUCOSE-109* UREA N-25* CREAT-0.3* SODIUM-138 POTASSIUM-3.4 CHLORIDE-105 TOTAL CO2-27 ANION GAP-9 ___ 10: 25PM estGFR-Using this ___ 10: 25PM ALT(SGPT)-100* AST(SGOT)-114* ALK PHOS-114* TOT BILI-1.6* ___ 10: 25PM LIPASE-77* ___ 10: 25PM ALBUMIN-3.3* ___ 10: 25PM WBC-5.0# RBC-4.29 HGB-14.3 HCT-42.6 MCV-99* MCH-33.3* MCHC-33.5 RDW-15.7* ___ 10: 25PM NEUTS-70.3* LYMPHS-16.5* MONOS-8.1 EOS-4.2* BASOS-0.8 ___ 10: 25PM PLT COUNT-71* ___ 10: 25PM ___ PTT-30.9 ___ ___ 10: 25PM ___ . CXR: No acute cardiopulmonary process. U/S: 1. Nodular appearance of the liver compatible with cirrhosis. Signs of portal hypertension including small amount of ascites and splenomegaly. 2. Cholelithiasis. 3. Patent portal veins with normal hepatopetal flow. Diagnostic para attempted in the ED, unsuccessful. On the floor, pt c/o abd distension and discomfort. <MEDICATIONS ON ADMISSION> The Preadmission Medication list is accurate and complete. 1. Furosemide 20 mg PO DAILY 2. Spironolactone 50 mg PO DAILY 3. Albuterol Inhaler 2 PUFF IH Q4H: PRN wheezing, SOB 4. Raltegravir 400 mg PO BID 5. Emtricitabine-Tenofovir (Truvada) 1 TAB PO DAILY 6. Nicotine Patch 14 mg TD DAILY 7. Ipratropium Bromide Neb 1 NEB IH Q6H SOB <DISCHARGE MEDICATIONS> 1. Albuterol Inhaler 2 PUFF IH Q4H: PRN wheezing, SOB 2. Emtricitabine-Tenofovir (Truvada) 1 TAB PO DAILY 3. Furosemide 40 mg PO DAILY RX *furosemide 40 mg 1 tablet(s) by mouth Daily Disp #*30 Tablet Refills: *3 4. Ipratropium Bromide Neb 1 NEB IH Q6H SOB 5. Nicotine Patch 14 mg TD DAILY 6. Raltegravir 400 mg PO BID 7. Spironolactone 50 mg PO DAILY 8. Acetaminophen 500 mg PO Q6H: PRN pain <DISCHARGE DISPOSITION> Home <DISCHARGE DIAGNOSIS> Ascites from Portal HTN <DISCHARGE CONDITION> Mental Status: Clear and coherent. Level of Consciousness: Alert and interactive. Activity Status: Ambulatory - Independent. <FOLLOWUP INSTRUCTIONS> ___ <DISCHARGE INSTRUCTIONS> Dear Ms. ___, It was a pleasure taking care of you! You came to us with stomach pain and worsening distension. While you were here we did a paracentesis to remove 1.5L of fluid from your belly. We also placed you on you 40 mg of Lasix and 50 mg of Aldactone to help you urinate the excess fluid still in your belly. As we discussed, everyone has a different dose of lasix required to make them urinate and it\'s likely that you weren\'t taking a high enough dose. Please take these medications daily to keep excess fluid off and eat a low salt diet. You will follow up with Dr. ___ in liver clinic and from there have your colonoscopy and EGD scheduled. Of course, we are always here if you need us. We wish you all the best! Your ___ Team. '
cleaned_ehr = clean_ehr_text(raw_ehr)
print(cleaned_ehr)

['allergy', 'no', 'known', 'allergies', 'adverse', 'drug', 'reactions', 'chief', 'complaint', 'worsen', 'abd', 'distension', 'pain', 'major', 'surgical', 'invasive', 'procedure', 'paracentesis', 'history', 'present', 'illness', 'hcv', 'cirrhosis', 'c', 'b', 'ascite', 'hiv', 'art', 'h', 'o', 'ivdu', 'copd', 'bioplar', 'ptsd', 'present', 'osh', 'ed', 'worsen', 'abd', 'distension', 'past', 'week', 'pt', 'report', 'self', 'discontinue', 'lasix', 'spirnolactone', 'week', 'ago', 'feel', 'like', 'want', 'chemical', 'follow', 'na', 'restrict', 'diet', 'past', 'week', 'note', 'have', 'worsen', 'abd', 'distension', 'discomfort', 'deny', 'edema', 'sob', 'orthopnea', 'deny', 'f', 'c', 'n', 'v', 'd', 'c', 'dysuria', 'food', 'poisoning', 'week', 'ago', 'eat', 'stale', 'cake', 'n', 'v', '20', 'min', 'food', 'ingestion', 'resolve', 'day', 'deny', 'recent', 'illness', 'sick', 'contact', 'note', 'notice', 'gum', 'bleeding', 'brush', 'tooth', 'recent', 'week', 'deny', 'easy', 'bruising', 'melena', 'brbpr

In [ ]:
import pandas as pd
import re

# Define cardiac-related keywords
cardiac_keywords = [
    "heart", "cardiac", "cardiovascular", "myocardium", "pericardium", "atrium", "atria",
    "ventricle", "coronary", "ecg", "ekg", "electrocardiogram", "echo", "echocardiogram",
    "holter", "stress test", "troponin", "bnp", "nt-probnp", "myocardial infarction",
    "heart failure", "coronary artery disease", "cad", "arrhythmia", "bradycardia",
    "tachycardia", "atherosclerosis", "angina", "atrial fibrillation", "afib",
    "ventricular fibrillation", "hypertrophic cardiomyopathy", "dilated cardiomyopathy",
    "valvular heart disease", "congenital heart defect", "ischemic heart disease",
    "stent", "angioplasty", "pacemaker", "defibrillator", "cabg"
]

# Compile regex pattern
pattern = re.compile(r'\b(?:' + '|'.join(map(re.escape, cardiac_keywords)) + r')\b', re.IGNORECASE)

# Apply logic to the DataFrame
def label_cardiac(text):
    if pd.isnull(text):
        return "No cardiac disease mentioned"
    return text if pattern.search(text) else "No cardiac disease mentioned"

# Apply to the 'target' column
df['target'] = df['target'].apply(label_cardiac)


In [ ]:
df

,target,augmented_input
0,"___ HCV cirrhosis c/b ascites, hiv on ART, h/o...",<Age> 52 <Mortality> Alive <Note Type> DS <SEX...
1,"___ with HIV on HAART, HCV cirrhosis with asci...",<Age> 52 <Mortality> Alive <Note Type> DS <SEX...
2,No cardiac disease mentioned,<Age> 48 <Mortality> Deceased <Note Type> DS <...
3,No cardiac disease mentioned,<Age> 48 <Mortality> Deceased <Note Type> DS <...
4,No cardiac disease mentioned,<Age> 34 <Mortality> Deceased <Note Type> DS <...
...,...,...
270028,No cardiac disease mentioned,<Age> 57 <Mortality> Deceased <Note Type> DS <...
270029,No cardiac disease mentioned,<Age> 46 <Mortality> Deceased <Note Type> DS <...
270030,No cardiac disease mentioned,<Age> 58 <Mortality> Alive <Note Type> DS <SEX...
270031,Mr. ___ was admitted with status epilepticus. ...,<Age> 58 <Mortality> Alive <Note Type> DS <SEX...


In [ ]:
from tqdm import tqdm
tqdm.pandas()

def clean_ehr_text(raw_text):
    if pd.isnull(raw_text):
        return []

    remove_sections = [
        r"<SEX>.*?(?=<|$)", r"<SERVICE>.*?(?=<|$)", r"<Age>.*?(?=<|$)", r"<Mortality>.*?(?=<|$)",
        r"<Note Type>.*?(?=<|$)", r"<ATTENDING>.*?(?=<|$)", r"<SOCIAL HISTORY>.*?(?=<|$)",
        r"<FAMILY HISTORY>.*?(?=<|$)", r"<DISCHARGE INSTRUCTIONS>.*?(?=<|$)",
        r"<DISCHARGE DISPOSITION>.*?(?=<|$)", r"<DISCHARGE DIAGNOSIS>.*?(?=<|$)",
        r"<DISCHARGE CONDITION>.*?(?=<|$)", r"<FOLLOWUP INSTRUCTIONS>.*?(?=<|$)",
        r"<DISCHARGE MEDICATIONS>.*?(?=<|$)", r"<PAST MEDICAL HISTORY>.*?(?=<|$)",
    ]

    combined_pattern = re.compile("|".join(remove_sections), re.IGNORECASE | re.DOTALL)
    cleaned_text = re.sub(combined_pattern, "", raw_text)

    # Format cleanup
    cleaned_text = re.sub(r'\n\s*\n+', '\n\n', cleaned_text).strip()
    text = cleaned_text
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"(</?[A-Z ]+>)", r"\n\1\n", text)
    text = re.sub(r"_+", "", text)
    text = re.sub(r"\d{1,2}:\d{2}\s*(?:AM|PM)?", "", text, flags=re.IGNORECASE)
    text = re.sub(r"[\.\:]\s*___", "", text)
    text = re.sub(r"(</?[A-Z ]+>)", lambda m: m.group(1).upper(), text)
    text = re.sub(r"\s+([,:;])", r"\1", text)
    text = re.sub(r"([,:;])\s+", r"\1 ", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"\n{2,}", "\n", text)

    # Tokenize and lemmatize
    doc = nlp(text.strip())
    tokens = [
        token.lemma_.lower().strip()
        for token in doc
        if (token.text.lower() in keep_words) or (not token.is_stop and token.text not in punctuations)
    ]
    return tokens


# ========== Step 4: Apply to DataFrame ==========
# Assuming your DataFrame is named df and contains 'target' and 'augmented_input'
df["target_tokens"] = df["target"].progress_apply(clean_ehr_text)
df["augmented_input_tokens"] = df["augmented_input"].progress_apply(clean_ehr_text)
df = df[["target_tokens", "augmented_input_tokens"]]

 50%|████▉     | 134194/270033 [1:29:44<1:31:16, 24.80it/s]